In [1]:
import _resume_eval_import_helper

In [2]:
import os 
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from uuid import uuid4
import pandas as pd
import json
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI 
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables.base import RunnableSequence
from prompts.two_stage_eval_jd import TWO_STAGE_EVAL_JD_PROMPT
from prompts.two_stage_eval_cv import TWO_STAGE_EVAL_CV_PROMPT

import logging
import time
from datetime import datetime 
from typing import Dict, Any, Union, List, Tuple

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

In [3]:
load_dotenv(find_dotenv("../../.env"))

True

In [4]:
# evaluate resume 
def two_stage_eval_jd(model_tuples: List[Tuple[str, RunnableSequence]], job_description: str, job_id: str, output_dir: str) -> Union[pd.DataFrame, None]:
    model_results = {}
    for model_name, grader in model_tuples:
        try:
            result = grader.invoke({"job_description": job_description})
            model_results[model_name] = result

            # save model result 
            json_file = os.path.join(output_dir, f"{job_id}_{model_name}.json")
            with open(json_file, "w") as f:
                json.dump(result, f, indent=4)
            time.sleep(2.1)  # Add a small delay to avoid rate limiting

        except Exception as e:
            error_msg = f"Error with {model_name} for job_id: {job_id}. Error: {str(e)}"
            logging.error(error_msg)
            print(error_msg)

    if not model_results:
        error_msg = f"All models failed for job_id: {job_id}."
        logging.error(error_msg)
        print(error_msg)
        return None
    
def two_stage_eval_cv(model_tuples: List[Tuple[str, RunnableSequence]], job_requirements: str, job_id: str, cv: str, cv_id: str, output_dir: str) -> Union[pd.DataFrame, None]:
    model_results = {}
    for model_name, grader in model_tuples:
        try:
            result = grader.invoke({"job_requirements": job_requirements, "resume": cv})
            model_results[model_name] = result

            # save model result 
            json_file = os.path.join(output_dir, f"{job_id}_{cv_id}_{model_name}.json")
            with open(json_file, "w") as f:
                json.dump(result, f, indent=4)
            time.sleep(2.1)  # Add a small delay to avoid rate limiting

        except Exception as e:
            error_msg = f"Error with {model_name} for job_id: {job_id}. Error: {str(e)}"
            logging.error(error_msg)
            print(error_msg)

    if not model_results:
        error_msg = f"All models failed for job_id: {job_id}."
        logging.error(error_msg)
        print(error_msg)
        return None
    

# global variables


In [5]:
# get current time 
current_time = datetime.now().strftime(("%Y%m%d_%H%M"))
output_dir = f"./output_{current_time}/"

# create output directory 
os.makedirs(output_dir, exist_ok=True)

temperature = 0
max_tokens = 2048

# set up logger 
log_file = os.path.join(output_dir, "evaluation_log.txt")
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(message)s", level=logging.INFO, filename=log_file, datefmt="%Y-%m-%d %H:%M:%S")

# import dataset


In [6]:
jobs = pd.read_csv(os.path.join("output", "filtered_job_description.csv"))

# 1st stage: jd evaluation


In [7]:
groq_llm = ChatGroq(model="llama3-70b-8192", temperature=temperature, max_tokens=max_tokens)
gpt_llm = ChatOpenAI(model="chatgpt-4o-latest", temperature=temperature, max_tokens=max_tokens)
# anthropic_llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=temperature, max_tokens=max_tokens)
# llama3_llm = ChatOllama(model="llama3", temperature=temperature, max_tokens=max_tokens)
 
jd_eval_prompt = PromptTemplate(
    input_variables=["job_description"],
    template=TWO_STAGE_EVAL_JD_PROMPT
    )

gpt_grader = jd_eval_prompt | gpt_llm | JsonOutputParser()
groq_grader = jd_eval_prompt | groq_llm | JsonOutputParser()
# anthropic_grader = resume_eval_prompt | anthropic_llm | JsonOutputParser()
# llama3_grader = resume_eval_prompt | llama3_llm | JsonOutputParser()

model_tuples = [
    # ("gpt4o", gpt_grader),
    # ("anthropic", anthropic_grader),
    # ("llama3", llama3_grader)
    ("groq", groq_grader)
]


In [8]:
def process_jobs(jod_data):
    job_id, job_description = jod_data
    return two_stage_eval_jd(model_tuples, job_description, job_id, output_dir)

def process_all_jobs():
    job_data = jobs[["Job ID", "Job Description"]].values
    # cv_data = talent_pool[["ID", "Resume"]].values

    # total_pairs = len(job_data) * len(cv_data)
    total_jobs = len(job_data)
    
    with ThreadPoolExecutor(max_workers=1) as executor:
        futures = [] 
        for job in job_data:
            futures.append(executor.submit(process_jobs, job))
            
        for future in tqdm(as_completed(futures), total=total_jobs, desc="Processing all jobs"):
            try:
                future = future.result()
            except Exception as e:
                print(f"Error: {e}")


In [9]:
process_all_jobs()

Processing job-cv pairs:   0%|          | 0/15 [00:00<?, ?it/s]

# 2nd stage: resume evaluation


In [10]:
# prepare data

job_data = []

for file in Path("output_20240901_2335").glob("*.json"):
        file_name = file.stem
        job_id = file_name.split("_")[0]
        model_name = file_name.split("_")[1]
        with open(file, "r") as f:
            job_description = json.load(f)
            
            job_data.append((job_id, job_description))

In [11]:
talent_pool = pd.read_csv(os.path.join("output", "filtered_talent_pool.csv"))

In [14]:
groq_llm = ChatGroq(model="llama3-70b-8192", temperature=temperature, max_tokens=max_tokens)

cv_eval_prompt = PromptTemplate(
    input_variables=["job_requirements", "resume"],
    template=TWO_STAGE_EVAL_CV_PROMPT
    )

groq_grader = cv_eval_prompt | groq_llm | JsonOutputParser()

model_tuples = [
    # ("gpt4o", gpt_grader),
    # ("anthropic", anthropic_grader),
    # ("llama3", llama3_grader)
    ("groq", groq_grader)
]


In [15]:
def process_job_cv_pairs(jod_data, cv_data):
    job_id, job_requirements = jod_data
    cv_id, cv = cv_data
    return two_stage_eval_cv(model_tuples, job_requirements, job_id, cv, cv_id, output_dir)

def process_all_pairs():
    
    cv_data = talent_pool[["ID", "Resume"]].values
    total_pairs = len(job_data) * len(cv_data)
    
    with ThreadPoolExecutor(max_workers=1) as executor:
        futures = [] 
        for job in job_data:
            for cv in cv_data:
                futures.append(executor.submit(process_job_cv_pairs, job, cv))
            
        for future in tqdm(as_completed(futures), total=total_pairs, desc="Processing job-cv pairs"):
            try:
                future = future.result()
            except Exception as e:
                print(f"Error: {e}")


In [16]:
process_all_pairs()

Processing job-cv pairs:   0%|          | 0/705 [00:00<?, ?it/s]

# Evaluation


In [65]:
results = []
errors = []

for file in Path(output_dir).glob("*.json"):
    try:
        file_name = file.stem
        job_id, cv_id, model_name = file_name.split("_")
        with open(file, "r") as f:
            result = json.load(f)
        
        data = {
            "job_id": job_id,
            "cv_id": cv_id,
            "model_name": model_name,
            "original_technical_skills": result["resume_evaluation"]["original_scores"].get("technical_skills", None),
            "original_soft_skills": result["resume_evaluation"]["original_scores"].get("soft_skills", None),
            "original_experience": result["resume_evaluation"]["original_scores"].get("experience", None),
            "original_education": result["resume_evaluation"]["original_scores"].get("education", None),
            "recalibrated_technical_skills": result["recalibrated_scores"].get("technical_skills", None),
            "recalibrated_soft_skills": result["recalibrated_scores"].get("soft_skills", None),
            "recalibrated_experience": result["recalibrated_scores"].get("experience", None),
            "recalibrated_education": result["recalibrated_scores"].get("education", None),
            "inferred_experience": ", ".join(result["deeper_analysis"].get("inferred_experience", [])),
            "suitability": result["assessment"].get("suitability", None),
            "strengths": result["assessment"].get("strengths", None),
            "concerns": result["assessment"].get("concerns", None)
        }
        
        results.append(data)
    except Exception as e:
        errors.append({"file": str(file), "error": str(e)})
        print(f"Error processing {file}: {e}")

# Convert results to a DataFrame
df = pd.DataFrame(results)


In [66]:
df.isna().sum()

job_id                           0
cv_id                            0
model_name                       0
original_technical_skills        0
original_soft_skills             0
original_experience              0
original_education               0
recalibrated_technical_skills    0
recalibrated_soft_skills         0
recalibrated_experience          0
recalibrated_education           0
inferred_experience              0
suitability                      0
strengths                        0
concerns                         0
dtype: int64

In [67]:
df.columns

Index(['job_id', 'cv_id', 'model_name', 'original_technical_skills',
       'original_soft_skills', 'original_experience', 'original_education',
       'recalibrated_technical_skills', 'recalibrated_soft_skills',
       'recalibrated_experience', 'recalibrated_education',
       'inferred_experience', 'suitability', 'strengths', 'concerns'],
      dtype='object')

## calculate the fit score


In [68]:
# Define weights
weights = {
    "technical_skills": 0.6,
    "soft_skills": 0.1,
    "experience": 0.2,
    "education": 0.1
}

# Define score types
score_types = ["original", "recalibrated"]

# Calculate overall scores using pandas' dot product
for score_type in score_types:
    columns = [f"{score_type}_{skill}" for skill in weights.keys()]
    df[score_type+"_overall_score"] = df[columns].values.dot(pd.Series(weights).values)

In [84]:
df.groupby(["suitability", "job_id"])[["original_overall_score",	"recalibrated_overall_score"]].mean().sort_values(by="original_overall_score", ascending=False)

original_overall_score  \
suitability job_id                                                         
yes         5535b3b6-f919-4e04-bb23-5eb63436941f               81.833333   
            40705682-6752-41f0-8a6d-b01b9d7b1746               80.000000   
            b04cc5ce-b93e-427e-9439-0965e64779ff               80.000000   
            769c3093-32c5-4122-ae8f-d4f99a22354a               78.000000   
            ca0539ba-835b-48fb-bed0-705d6668c372               76.000000   
kiv         2f1247bd-4c98-4357-8b8b-ac52ee8698b2               67.266667   
            5535b3b6-f919-4e04-bb23-5eb63436941f               62.160000   
            40705682-6752-41f0-8a6d-b01b9d7b1746               61.107143   
            8f52ff7e-1929-49d9-86d3-052e98986b34               59.818182   
            769c3093-32c5-4122-ae8f-d4f99a22354a               57.285714   
            31fde0c0-866a-48f9-b399-aad6f9c472a8               54.357143   
            ca0539ba-835b-48fb-bed0-705d6668c372               53.870968   
            7e259146-d9df-411e-8550-2fef51dc7bc9               53.428571   
            dcb1513a-460d-46f7-81b1-23294a691bfd               52.666667   
            91793843-4ca9-4eb6-9d84-8c59a3d19812               52.625000   
            90c1ee72-9c8d-46d4-a671-cb7cba7f75df               49.727273   
            66c5e115-cf87-4917-9cff-c2d72156cff6               48.062500   
            83d835b5-ab57-4044-ace8-a5d8ffc0e254               47.000000   
            fb3ec592-b093-46ca-8dfd-de3e164b49ac               46.363636   
            b04cc5ce-b93e-427e-9439-0965e64779ff               45.100000   
no          769c3093-32c5-4122-ae8f-d4f99a22354a               36.480000   
            8f52ff7e-1929-49d9-86d3-052e98986b34               33.280000   
            b04cc5ce-b93e-427e-9439-0965e64779ff               32.923077   
            2f1247bd-4c98-4357-8b8b-ac52ee8698b2               32.411765   
            40705682-6752-41f0-8a6d-b01b9d7b1746               31.111111   
            dcb1513a-460d-46f7-81b1-23294a691bfd               30.758621   
            7e259146-d9df-411e-8550-2fef51dc7bc9               30.325000   
            5535b3b6-f919-4e04-bb23-5eb63436941f               29.375000   
            fb3ec592-b093-46ca-8dfd-de3e164b49ac               28.250000   
            ca0539ba-835b-48fb-bed0-705d6668c372               25.800000   
            91793843-4ca9-4eb6-9d84-8c59a3d19812               25.483871   
            90c1ee72-9c8d-46d4-a671-cb7cba7f75df               24.750000   
            31fde0c0-866a-48f9-b399-aad6f9c472a8               23.818182   
            83d835b5-ab57-4044-ace8-a5d8ffc0e254               23.000000   
            66c5e115-cf87-4917-9cff-c2d72156cff6               21.967742   

                                                  recalibrated_overall_score  
suitability job_id                                                            
yes         5535b3b6-f919-4e04-bb23-5eb63436941f                   86.666667  
            40705682-6752-41f0-8a6d-b01b9d7b1746                   84.500000  
            b04cc5ce-b93e-427e-9439-0965e64779ff                   85.000000  
            769c3093-32c5-4122-ae8f-d4f99a22354a                   82.500000  
            ca0539ba-835b-48fb-bed0-705d6668c372                   81.000000  
kiv         2f1247bd-4c98-4357-8b8b-ac52ee8698b2                   74.233333  
            5535b3b6-f919-4e04-bb23-5eb63436941f                   69.456000  
            40705682-6752-41f0-8a6d-b01b9d7b1746                   67.892857  
            8f52ff7e-1929-49d9-86d3-052e98986b34                   66.772727  
            769c3093-32c5-4122-ae8f-d4f99a22354a                   65.642857  
            31fde0c0-866a-48f9-b399-aad6f9c472a8                   62.571429  
            ca0539ba-835b-48fb-bed0-705d6668c372                   61.016129  
            7e259146-d9df-411e-8550-2fef51dc7bc9                   60.428571  
            dcb1513a-460d-46f7-81b1-23294a691bfd    

In [85]:
df.iloc[694]
# # Save results to CSV

job_id                                        2f1247bd-4c98-4357-8b8b-ac52ee8698b2
cv_id                                         3e688d21-39ee-4601-9fc8-b74d9a359063
model_name                                                                    groq
original_technical_skills                                                       80
original_soft_skills                                                            60
original_experience                                                             80
original_education                                                             100
recalibrated_technical_skills                                                   90
recalibrated_soft_skills                                                        70
recalibrated_experience                                                         90
recalibrated_education                                                         100
inferred_experience              strong understanding of SAP Business Intellige...
suit

Detailed results saved to output/two_stage_evaluation_results_detailed.csv


# merge with job description and talent pool


In [89]:
df.columns

Index(['job_id', 'cv_id', 'model_name', 'original_technical_skills',
       'original_soft_skills', 'original_experience', 'original_education',
       'recalibrated_technical_skills', 'recalibrated_soft_skills',
       'recalibrated_experience', 'recalibrated_education',
       'inferred_experience', 'suitability', 'strengths', 'concerns',
       'original_overall_score', 'recalibrated_overall_score'],
      dtype='object')

In [90]:
job_description.columns, talent_pool.columns

(Index(['Job Title', 'Job Description', 'Job ID'], dtype='object'),
 Index(['Category', 'Resume', 'ID'], dtype='object'))

In [ ]:
job_description = pd.read_csv("output/filtered_job_description.csv")

In [94]:
job_pool = pd.read_csv("output/filtered_job_description.csv")
talent_pool = pd.read_csv("output/filtered_talent_pool.csv")

job_pool.rename(columns={"Job ID": "job_id", "Job Description": "job_description", "Job Title": "job_title"}, inplace=True)
talent_pool.rename(columns={"ID": "cv_id", "Resume": "cv", "Category": "cv_category"}, inplace=True)

In [95]:
df = pd.merge(df, job_pool, on=["job_id"], how="left")
df = pd.merge(df, talent_pool, on=["cv_id"], how="left")

In [102]:
from pprint import pprint

for col in df.columns:
    pprint(col)
    pprint(df.iloc[694][col])
    print("-"*100)


'job_id'
'2f1247bd-4c98-4357-8b8b-ac52ee8698b2'
----------------------------------------------------------------------------------------------------
'cv_id'
'3e688d21-39ee-4601-9fc8-b74d9a359063'
----------------------------------------------------------------------------------------------------
'model_name'
'groq'
----------------------------------------------------------------------------------------------------
'original_technical_skills'
80
----------------------------------------------------------------------------------------------------
'original_soft_skills'
60
----------------------------------------------------------------------------------------------------
'original_experience'
80
----------------------------------------------------------------------------------------------------
'original_education'
100
----------------------------------------------------------------------------------------------------
'recalibrated_technical_skills'
90
------------------------------------

In [105]:
df_ = df[df["suitability"]=="yes"].reset_index(drop=False)

for _, row in df_.iterrows():
    for col in df_.columns:
        pprint(col)
        pprint(row[col])
        print("-"*50)
    
    print()
    print("="*100)
    print()

'index'
36
--------------------------------------------------
'job_id'
'5535b3b6-f919-4e04-bb23-5eb63436941f'
--------------------------------------------------
'cv_id'
'b2787176-a5e0-45be-affa-6f955925ddc6'
--------------------------------------------------
'model_name'
'groq'
--------------------------------------------------
'original_technical_skills'
80
--------------------------------------------------
'original_soft_skills'
60
--------------------------------------------------
'original_experience'
80
--------------------------------------------------
'original_education'
100
--------------------------------------------------
'recalibrated_technical_skills'
85
--------------------------------------------------
'recalibrated_soft_skills'
65
--------------------------------------------------
'recalibrated_experience'
85
--------------------------------------------------
'recalibrated_education'
100
--------------------------------------------------
'inferred_experience'
('Experien

In [106]:
# Save results to CSV
csv_file = os.path.join("output/two_stage_evaluation_results_detailed.csv")
df.to_csv(csv_file, index=False)

print(f"Detailed results saved to {csv_file}")

# # Save errors to CSV if any occurred
# if errors:
#     error_df = pd.DataFrame(errors)
#     error_csv = os.path.join(output_dir, "processing_errors.csv")
#     error_df.to_csv(error_csv, index=False)
#     print(f"Errors saved to {error_csv}")

Detailed results saved to output/two_stage_evaluation_results_detailed.csv
